In [1]:
import sys
import os

# Add the marketsim directory to the path
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_dir)

# Add the simulator's directory to the path so its relative imports work
simulator_dir = os.path.join(root_dir, "marketsim", "simulator")
parent_dir = os.path.dirname(simulator_dir)  # This is the marketsim directory
sys.path.append(parent_dir)

print(f"Added to path: {root_dir}")
print(f"Added parent directory to path: {parent_dir}")



from marketsim.simulator.melo_simulator import MELOSimulatorSampledArrival
from marketsim.egta.game import AbstractGame
from marketsim.egta.symmetric_game import SymmetricGame
from marketsim.egta.reductions.dpr import DPRGAME
from marketsim.egta.process_data import create_symmetric_game_from_data
from marketsim.egta.utils.eq_computation import find_equilibria
from marketsim.egta.utils.log_multimodal import logmultinomial
from marketsim.egta.utils.random_functions import *
from marketsim.egta.utils.simplex_operations import *
from marketsim.egta.schedulers.melo_scheduler import MeloScheduler

import numpy as np
import torch

Matplotlib created a temporary cache directory at /var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/matplotlib-lagmrzdk because the default path (/Users/gabesmithline/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Added to path: /Users/gabesmithline/Desktop/SRG/melo_project
Added parent directory to path: /Users/gabesmithline/Desktop/SRG/melo_project/marketsim


In [2]:
import itertools
from typing import List, Dict, Any, Tuple

def generate_profiles(strategies: List[Dict[str, Any]], num_players: int) -> List[List[Dict[str, Any]]]:
    """
    Generate all possible strategy profiles for a symmetric game.
    
    Parameters:
    -----------
    strategies : List[Dict[str, Any]]
        List of strategy dictionaries, each containing at least 'name' and 'param'
    num_players : int
        Number of players in the game
        
    Returns:
    --------
    List[List[Dict[str, Any]]]
        List of strategy profiles, where each profile is a list of strategy dicts (one per player)
    """
    # Generate all possible combinations of strategies with replacement
    # This gives us counts of each strategy (e.g., 3 of strategy 0, 2 of strategy 1, etc.)
    strategy_combinations = itertools.combinations_with_replacement(range(len(strategies)), num_players)
    
    profiles = []
    
    for combination in strategy_combinations:
        # Convert the combination (which has indices) to a list of actual strategy objects
        profile = [strategies[idx] for idx in combination]
        profiles.append(profile)
    
    return profiles


def generate_profiles_counts(strategies: List[Dict[str, Any]], num_players: int) -> List[Tuple[List[Dict[str, Any]], List[int]]]:
    """
    Generate all possible strategy profiles with counts for a symmetric game.
    
    This version returns both the unique strategies and their counts in each profile,
    which can be more efficient for some EGTA algorithms.
    
    Parameters:
    -----------
    strategies : List[Dict[str, Any]]
        List of strategy dictionaries, each containing at least 'name' and 'param'
    num_players : int
        Number of players in the game
        
    Returns:
    --------
    List[Tuple[List[Dict[str, Any]], List[int]]]
        List of (unique_strategies, counts) tuples for each profile
    """
    from collections import Counter
    
    # Generate all possible combinations of strategies with replacement
    strategy_combinations = itertools.combinations_with_replacement(range(len(strategies)), num_players)
    
    profiles_with_counts = []
    
    for combination in strategy_combinations:
        # Count occurrences of each strategy
        counts = Counter(combination)
        
        # Create lists of unique strategies and their counts
        unique_strategies = [strategies[idx] for idx in counts.keys()]
        strategy_counts = list(counts.values())
        
        profiles_with_counts.append((unique_strategies, strategy_counts))
    
    return profiles_with_counts

In [3]:
# Set up random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

def run_initial_simulations(strategies: List[Dict[str, Any]], num_players: int, 
                          num_runs: int = 5, sim_time: int = 1000) -> List[List[Tuple]]:
    """
    Run initial simulations for all possible strategy profiles.
    
    Parameters:
    -----------
    strategies : List[Dict[str, Any]]
        List of strategies to explore
    num_players : int
        Number of players per simulation
    num_runs : int
        Number of simulation runs per profile (for statistical robustness)
    sim_time : int
        Simulation time steps
        
    Returns:
    --------
    List[List[Tuple]]
        List of raw data from simulation runs
    """
    # Generate all possible profiles
    profiles = generate_profiles(strategies, num_players)
    
    # Store simulation results
    all_results = []
    
    print(f"Running {len(profiles)} profiles with {num_runs} repetitions each...")
    
    # For each profile
    for profile_idx, profile in enumerate(profiles):
        print(f"Simulating profile {profile_idx+1}/{len(profiles)}: " + 
              ", ".join([f"{s['name']}" for s in profile]))
        
        # Run multiple times for statistical robustness
        for run in range(num_runs):
            # Configure simulator
            simulator = MELOSimulatorSampledArrival(
                num_background_agents=num_players,
                sim_time=sim_time,
                num_assets=1,
                lam=0.1,  # Default lambda
                mean=100,
                r=0.05,
                shock_var=10,
                q_max=10,
                pv_var=5e6,
                eta=0.2
            )
            
            # Set up agents with their strategies
            for agent_id, strategy in enumerate(profile):
                # Create agent with the appropriate strategy
                # Note: MELOSimulatorSampledArrival doesn't have a set_agent_strategy method
                # Instead, we need to configure agents correctly during initialization
                
                # Since the agents are created in the simulator's __init__, we need to
                # modify the agent's lambda parameter after creation
                # Access the agent and configure its lambda parameter
                if agent_id < len(simulator.agents):
                    agent = simulator.agents[agent_id]
                    if hasattr(agent, 'lam'):
                        agent.lam = strategy['param']
                    else:
                        print(f"Warning: Agent {agent_id} doesn't have a 'lam' attribute")
            
            # Run simulation
            simulator.run()
            
            # Collect payoffs
            # This assumes the simulator returns agent profits at the end
            # We may need to modify this based on how payoffs are accessed
            payoffs = []
            for agent_id in range(num_players):
                if agent_id in simulator.agents:
                    # Get agent's profit - this will depend on how your simulator stores this
                    # You might need to access agent.meloProfit or similar
                    if hasattr(simulator.agents[agent_id], 'meloProfit'):
                        payoff = simulator.agents[agent_id].meloProfit
                    else:
                        # Fallback - use some other profit measure if available
                        position = simulator.agents[agent_id].position
                        cash = simulator.agents[agent_id].cash
                        # Estimate payoff based on final position and cash
                        fundamental_val = simulator.market.get_final_fundamental()
                        payoff = position * fundamental_val + cash
                    
                    payoffs.append(payoff)
                else:
                    payoffs.append(0.0)  # Default if agent not found
            
            # Store profile and payoffs
            profile_result = []
            for agent_id, (strategy, payoff) in enumerate(zip(profile, payoffs)):
                profile_result.append((agent_id, strategy["name"], payoff))
            
            all_results.append(profile_result)
    
    return all_results

In [4]:
# Define initial strategies to explore
initial_strategies = [
        {"name": "MELO_0.1", "param": 0.1},  # Strategy with lambda = 0.1
        {"name": "MELO_0.2", "param": 0.2}   # Strategy with lambda = 0.2
    ]
    
    # Define full strategy space (for later expansion in Quiesce)
strategy_space = [
        {"name": f"MELO_{lamb:.1f}", "param": lamb} 
        for lamb in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    ]
    
    # Parameters for simulation
num_players = 3  # Small number for initial test
num_runs = 3     # Multiple runs per profile for statistical significance
    
    # Run initial simulations
simulation_results = run_initial_simulations(
        strategies=initial_strategies,
        num_players=num_players,
        num_runs=num_runs
    )
    
    # Process results into a SymmetricGame
symmetric_game = create_symmetric_game_from_data(simulation_results)

# Print game information
print(f"Game created with {symmetric_game.num_players} players and {symmetric_game.num_actions} strategies")
print(f"Strategy names: {symmetric_game.strategy_names}")

# To print the payoff table
if hasattr(symmetric_game, 'print_full_heuristic_payoff_table'):
    symmetric_game.print_full_heuristic_payoff_table()
else:
    print("Payoff matrix shape:", symmetric_game.payoff_table.shape)

Running 4 profiles with 3 repetitions each...
Simulating profile 1/4: MELO_0.1, MELO_0.1, MELO_0.1
At the end of the simulation we get {-12: tensor(0.), -11: tensor(0.), -10: tensor(0.), -9: tensor(0.), -8: tensor(0.), -7: tensor(0.), -6: tensor(0.), -5: tensor(0.), -4: tensor(0.), -3: tensor(0.), -2: tensor(0.), -1: tensor(0.), 0: tensor(0.), 1: tensor(0.), 2: tensor(0.), 3: tensor(0.)}
MELO_ At the end of the simulation we get {-12: 0, -11: 0, -10: 0, -9: 0, -8: 0, -7: 0, -6: 0, -5: 0, -4: 0, -3: 0, -2: 0, -1: 0, 0: 0, 1: 0, 2: 0, 3: 0}
At the end of the simulation we get {-12: tensor(0.), -11: tensor(0.), -10: tensor(0.), -9: tensor(0.), -8: tensor(0.), -7: tensor(0.), -6: tensor(0.), -5: tensor(0.), -4: tensor(0.), -3: tensor(0.), -2: tensor(0.), -1: tensor(0.), 0: tensor(0.), 1: tensor(0.), 2: tensor(0.), 3: tensor(0.)}
MELO_ At the end of the simulation we get {-12: 0, -11: 0, -10: 0, -9: 0, -8: 0, -7: 0, -6: 0, -5: 0, -4: 0, -3: 0, -2: 0, -1: 0, 0: 0, 1: 0, 2: 0, 3: 0}
At the en

/Users/gabesmithline/Desktop/SRG/melo_project/marketsim/egta/symmetric_game.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.config_table = torch.tensor(config_table, dtype=torch.float32, device=device)
/Users/gabesmithline/Desktop/SRG/melo_project/marketsim/egta/symmetric_game.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.payoff_table = torch.tensor(payoff_table, dtype=torch.float32, device=device)


In [5]:
symmetric_game.print_full_heuristic_payoff_table()

| Profile   |   MELO_0.1 Count |   MELO_0.2 Count | MELO_0.1 Payoff   | MELO_0.2 Payoff   |
|:----------|-----------------:|-----------------:|:------------------|:------------------|
| Profile 1 |                3 |                0 | 0.0000            | N/A               |
| Profile 2 |                2 |                1 | 0.0000            | 0.0000            |
| Profile 3 |                1 |                2 | 0.0000            | 0.0000            |
| Profile 4 |                0 |                3 | N/A               | 0.0000            |


In [ ]:
# Create scheduler to handle on-demand simulations during Quiesce
scheduler = (
    strategy_names=strategy_names,
    num_players=symmetric_game.num_players,
    simulator_path="python -m marketsim.simulator.melo_simulator",
    simulator_config={
        "num_background_agents": 100,
        "sim_time": 1000
        # Other simulator parameters
    }
)
